In [ ]:
#Aazir

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Restart runtime after installing if prompted by Colab
!pip install -q "torch" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q monai
!pip install -q nibabel scikit-image pandas tqdm SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 16.4 MB/s eta 0:00:00


In [ ]:
import os, json, glob

root = "/content/drive/MyDrive/kits23/dataset"   # your root

case_folders = sorted([d for d in glob.glob(os.path.join(root, "case_*")) if os.path.isdir(d)])

datalist = {"training": []}

for case_path in case_folders:
    case_id = os.path.basename(case_path)
    # main label for the case
    label_path = os.path.join(case_path, "segmentation.nii.gz")

    # image path - if your CT images are inside a subfolder, adjust this
    # For example, if your CT is inside 'images/CT.nii.gz' for each case, update accordingly
    image_path = os.path.join(case_path, "image.nii.gz")  # <-- replace 'image.nii.gz' with your actual CT filename

    if not os.path.exists(label_path) or not os.path.exists(image_path):
        print(f"Skipping {case_id}, missing image or label")
        continue

    datalist["training"].append({
        "image": os.path.relpath(image_path, start=root),
        "label": os.path.relpath(label_path, start=root),
        "image_filename": os.path.basename(image_path)
    })

dataset_json = {
    "name": "KiTS23_custom",
    "description": "KiTS23 formatted dataset from per-case folders",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "modality": {"0": "CT"},
    "labels": {"0": "background", "1": "kidney_and_mass", "2": "mass", "3": "tumor"},
    "numTraining": len(datalist["training"]),
    "training": datalist["training"],
    "test": []
}

with open(os.path.join(root, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=2)

print("dataset.json created with", len(datalist["training"]), "cases")
